Common gotchas with OCP
---------------------------------

# OutOfMemoryError

If you see errors like:

```
torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 390.00 MiB (GPU 0; 10.76 GiB total capacity; 9.59 GiB already allocated; 170.06 MiB free; 9.81 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
```

It means your GPU is out of memory. Some reasons could be that you have multiple notebooks open that are using the GPU, e.g. they have loaded a calculator or something. Try closing all the other notebooks. 

It could also mean the batch size is too large to fit in memory. You can try making it smaller in the yml config file (optim.batch_size).

If it is an option, you can try a GPU with more memory, or you may be able to split the job over multiple GPUs.

# I want the energy of a gas phase atom

But I get an error like 

```
RuntimeError: cannot reshape tensor of 0 elements into shape [0, -1] because the unspecified dimension size -1 can be any value and is ambiguous
```

The problem here is that no neighbors are found for the single atom which causes an error. This may be model dependent. There is currently no way to get atomic energies for some models.

In [2]:
%%capture
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
cp = "gnoc_oc22_oc20_all_s2ef.pt"
calc = OCPCalculator(checkpoint=cp)

In [3]:
from ase.build import bulk
atoms = bulk('Cu', a=10)
atoms.set_calculator(calc)
atoms.get_potential_energy()

RuntimeError: cannot reshape tensor of 0 elements into shape [0, -1] because the unspecified dimension size -1 can be any value and is ambiguous